Notes:

Source of BLS data: https://www.bls.gov/oes/tables.htm


## Purpose
We have occupation employment numbers from the Bureau of Labor Statistics by metropolitan areas, but both our maps and CO2 emission factors are in zip code. 

Here we come up with a mapping of metropolitan and non-metropolitan areas listed in the BLS data set to zip code. 

We start by using a mapping of zip code to MSA provided by the Bureau of Labor Statisics. This works for most major metropolitan areas. Non-metropolitan areas, however, are not listed. In those cases, we maps zips that have not been assigned to an MSA to the sum of all non-metropolitan areas in that state.

In [1]:
import pandas as pd
import numpy as np
import re
import sqlite3

Function to turn strings into a floating point

In [2]:
def make_flt(x):
    o = re.search('^[0-9\.]*$',str(x))
    if o:
        out = float(str(x))
    else:
        out = np.nan
    return out

First we are going to cull the zips in the DOL data set because it turns out they use postal zip codes as opposed to zip code tabulaton area (ZTCA) which is what we will need in R. In general, there is a pretty good one-to-one correspondance between the two. Here we will drop ZIPs that are not in our ZTCA data set.

Read in zips that are present in our R choropleth mapping object

In [3]:
zips = pd.read_csv('Datasets/zips.csv',index_col=0)

Get the set of zips

In [4]:
zips_in_r_set = set(zips['x'])

Now load in the mappings of zip to MSA that we have from DOL

In [6]:
zip_to_msa_og = pd.read_excel('Datasets/fs15_gpci_by_msa-ZIP.xls', sheet_name = 'fs15gpci by ZIP, owcp', skiprows=10)

Drop some of the columns we will not need

In [7]:
zip_to_msa_og.drop(['GPCI','GPCI.1','GPCI.2','Unnamed: 8'],axis=1,inplace=True)

Don't need Puerto Rico

In [8]:
zip_to_msa_og = zip_to_msa_og[zip_to_msa_og['STATE'] != 'PR']

Now get the set of zips in the DOL data set

In [9]:
zips_w_msas_set = set(zip_to_msa_og['ZIP CODE'])

Get the intersection of zips

In [10]:
zips_inter = list(zips_w_msas_set.intersection(zips_in_r_set))

Turn into a dataframe

In [11]:
good_zips = pd.DataFrame({'zip':zips_inter})

Do a merge to get the subset of elements in our DOL dataset that have an element in our mapping object

In [12]:
zip_to_msa = pd.merge(good_zips,zip_to_msa_og,left_on='zip',right_on = 'ZIP CODE')

Drop and sort

In [13]:
zip_to_msa.drop('zip',axis=1,inplace = True)

In [13]:
#zip_to_msa.sort(inplace = True)

In [14]:
zip_to_msa['id'] = zip_to_msa.index

Okay, let's start reading in data from BLS. First we are going to read in data for metropolitan areas.

i.e., Add the mapping of MSA to zipcode in MSA_M(year)_dl.xlsx

In [15]:
### --- 2012
a = pd.read_excel('Datasets/oesm12ma/MSA_M2012_dl_1_AK_IN.xls',sheet_name='MSA_dl_1')
b = pd.read_excel('Datasets/oesm12ma/MSA_M2012_dl_2_KS_NY.xls',sheet_name='MSA_dl_2')
c = pd.read_excel('Datasets/oesm12ma/MSA_M2012_dl_3_OH_WY.xls',sheet_name='MSA_dl_3')
oes = pd.concat([a,b,c], axis=0)
### ---

#oes = pd.read_excel('Datasets/oesm14ma/MSA_M2014_dl.xlsx',sheet_name='MSA_dl')
#oes = pd.read_excel('Datasets/oesm17ma/MSA_M2017_dl.xlsx',sheet_name='MSA_dl_1')
#oes = pd.read_excel('Datasets/oesm18ma/MSA_M2018_dl.xlsx',sheet_name='MSA_dl')

In [16]:
cols = oes.columns[0:7]

In [17]:
oes_lite = oes[cols]

In [18]:
#oes_lite['TOT_EMP'] = oes_lite['TOT_EMP'].apply(make_flt)
oes_lite.loc[:, 'TOT_EMP'] = oes_lite['TOT_EMP'].apply(make_flt)

/Users/nuoaleon/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


Do a merge with our zip_to_msa dataframe

In [19]:
out1 = pd.merge(zip_to_msa, oes_lite, left_on='MSA No.',right_on='AREA')

Next we are going to read in data from "aggregated" MSA regions that were broken out into smaller regions in the above dataset. We are then going to do a merge on our zip_to_msa dataframe

i.e., Add the mapping of MSA to zipcode in aMSA_M(year)_dl.xlsx


In [20]:
oes2 = None

try:
    oes2 = pd.read_excel('Datasets/oesm12ma/aMSA_M2012_dl.xls',sheet_name='AMSA_dl')
    #oes2 = pd.read_excel('Datasets/oesm14ma/aMSA_M2014_dl.xlsx',sheet_name='AMSA_dl')
    #oes2 = pd.read_excel('Datasets/oesm17ma/aMSA_M2017_dl.xlsx',sheet_name='AMSA_dl')
    #oes2 = pd.read_excel('Datasets/oesm18ma/aMSA_M2018_dl.xlsx',sheet_name='AMSA_dl')
except FileNotFoundError:
    pass


In [21]:
if oes2 is not None:
    oes_lite2 = oes2[cols]
else:
    oes_lite2 = None

In [22]:
if oes_lite2 is not None:
    #oes_lite2['TOT_EMP'] = oes_lite2['TOT_EMP'].apply(make_flt)
    oes_lite2.loc[:, 'TOT_EMP'] = oes_lite2['TOT_EMP'].apply(make_flt)

/Users/nuoaleon/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [23]:
if oes_lite2 is not None:
    out2 = pd.merge(zip_to_msa, oes_lite2,left_on='MSA No.',right_on='AREA')
else:
    out2 = None

Next there are a couple of MSAs whose names are out of date in the zip_to_msa dataframe

In [25]:
df_updated_msas = pd.read_excel('Datasets/Updated MSAs.xlsx')

/Users/nuoaleon/opt/anaconda3/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:296: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Figure out which metropolitan areas are missing

In [26]:
all_zips = set(zip_to_msa.index)

if out2 is not None:
    msa_zips = set(out1.id.values).union(set(out2.id.values))
else:
    msa_zips = set(out1.id.values)

missing_zips = list(all_zips.difference(msa_zips))
df_missing_zips = zip_to_msa.loc[missing_zips,:]
metro_missing = ~df_missing_zips['MSA Name'].str.contains('NONMETRO')
df_metro_missing = df_missing_zips.loc[metro_missing,:]

Now merge with the dataframe that has updated MSA names


i.e., get a mapping from old MSA to new MSA.

In [27]:
zip_to_msa_new = pd.merge(df_metro_missing,df_updated_msas.loc[:,['Old MSA','New MSA']],left_on = 'MSA No.',right_on = 'Old MSA')

In [28]:
zip_to_msa_new.drop(['MSA No.','Old MSA'],axis=1,inplace = True)
zip_to_msa_new.rename(columns = {'New MSA':'MSA No.'},inplace = True)

In [29]:
if oes_lite2 is not None:
    oes_lite3 = pd.concat([oes_lite, oes_lite2], axis=0)
else:
    oes_lite3 = oes_lite

In [30]:
#out3 = pd.merge(zip_to_msa_new, oes_lite, left_on='MSA No.',right_on='AREA')
out3 = pd.merge(zip_to_msa_new, oes_lite3, left_on='MSA No.',right_on='AREA')

In [31]:
all_zips = set(zip_to_msa.index)

if out2 is not None:
    msa_zips = (set(out1.id.values).union(set(out2.id.values))).union(set(out3.id.values))
else:
    msa_zips = set(out1.id.values).union(set(out3.id.values))

missing_zips = list(all_zips.difference(msa_zips))
df_missing_zips = zip_to_msa.loc[missing_zips,:]
metro_missing = ~df_missing_zips['MSA Name'].str.contains('NONMETRO')
df_metro_missing = df_missing_zips.loc[metro_missing,:]

Now we are going to assign all remaining non-metro zips to the non-metro data

i.e., Add the mapping of Non-MSA to zipcode in BOS_M(year)_dl.xlsx

In [32]:
oes3 = pd.read_excel('Datasets/oesm12ma/BOS_M2012_dl.xls',sheet_name='BOS_dl_1')
#oes3 = pd.read_excel('Datasets/oesm14ma/BOS_M2014_dl.xlsx',sheet_name='BOS_dl')
#oes3 = pd.read_excel('Datasets/oesm17ma/BOS_M2017_dl.xlsx',sheet_name='BOS_dl_1')
#oes3 = pd.read_excel('Datasets/oesm18ma/BOS_M2018_dl.xlsx',sheet_name='BOS_dl')

In [33]:
for col in oes3.columns[6:]:
    oes3[col] = oes3[col].apply(make_flt)

In [34]:
oes_lite3 = oes3[cols]

In [35]:
oes_lite3 = oes_lite3.groupby(['PRIM_STATE','OCC_TITLE','OCC_CODE','OCC_GROUP']).sum().reset_index()

In [36]:
oes_lite3['AREA_NAME'] = oes_lite3['PRIM_STATE']

In [37]:
out4 = pd.merge(df_missing_zips, oes_lite3, left_on = 'STATE',right_on='PRIM_STATE')
## Since here we join it using STATE, do we need to re-weight the TOP_EMP by zipcode count inside that state

In [38]:
# oes_lite3.head()
# out4[out4['ZIP CODE'] == 99547].head()
# out4[out4['ZIP CODE'] == 99546].head()

Now let's combine everything

In [39]:
if out2 is not None:
    out = pd.concat([out1,out2,out3[out1.columns],out4[out1.columns]],axis=0,ignore_index=True)
else:
    out = pd.concat([out1,out3[out1.columns],out4[out1.columns]],axis=0,ignore_index=True)

In [40]:
out.drop(['PRIM_STATE','MSA Name','County No.','AREA'],axis=1,inplace = True)

In [41]:
#out.sort('id',inplace=True)

We are going to write our data-set out as a SQL database to make it a little easier to access

In [43]:
def write_db(df,db):  
    '''
    Write dataframe to database. Assume we are appending to a database
    '''
    con = sqlite3.connect(db)
    df.to_sql('data',con,if_exists = 'replace')

In [42]:
# write_db(out,'Datasets/oes_may12_demo.db')

/Users/nuoaleon/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,
